# Week 3 — Part 03: Structured outputs (JSON) — parse + validate + retry/repair

**Estimated time:** 90–120 minutes

## Learning Objectives

- Parse model text into JSON safely
- Validate schema and separate parse vs schema failures
- Implement a capped retry/repair loop
- Save raw outputs for debugging


## Overview

Models can produce:

- valid JSON
- almost-JSON (single quotes, trailing commas, extra prose)

Downstream code needs a pass/fail signal. So we implement:

1. ask for strict JSON
2. parse it
3. validate schema
4. if invalid, retry with a repair prompt (capped)

Key habit: save raw output when parsing/validation fails so debugging is inspection, not guesswork.

In [ ]:
import json


def parse_json_strict(text: str) -> dict:
    data = json.loads(text)
    if not isinstance(data, dict):
        raise ValueError("expected a JSON object")
    return data


def validate_shape(data: dict) -> None:
    allowed = {"person", "company"}
    extra = set(data.keys()) - allowed
    missing = allowed - set(data.keys())
    if missing:
        raise ValueError(f"missing keys: {sorted(missing)}")
    if extra:
        raise ValueError(f"extra keys: {sorted(extra)}")

    for k in ["person", "company"]:
        v = data[k]
        if v is not None and not isinstance(v, str):
            raise ValueError(f"{k} must be string or null")


print(validate_shape(parse_json_strict('{"person": "Ada", "company": null}')))

In [ ]:
def parse_and_validate(text: str) -> dict:
    data = parse_json_strict(text)
    validate_shape(data)
    return data


bad_outputs = [
    "Here is the JSON: {\"person\": \"Ada\", \"company\": null}",
    "{'person': 'Ada', 'company': null}",
    '{"person": "Ada"}',
]

for raw in bad_outputs:
    try:
        parse_and_validate(raw)
        print("OK", raw)
    except Exception as e:
        print("FAIL", type(e).__name__, "->", str(e))

In [ ]:
def call_llm_stub(prompt: str) -> str:
    # Simulate a model that sometimes returns almost-JSON.
    if "REPAIR" in prompt:
        return '{"person": "Ada Lovelace", "company": null}'
    return "Here is the JSON: {\"person\": \"Ada Lovelace\", \"company\": null}"

In [ ]:
def extract_with_repair(text: str, call_llm, *, max_retries: int = 2) -> dict:
    base_prompt = (
        "Return ONLY JSON with keys person, company (null when unknown).\n"
        f"INPUT:\n{text}\n"
    )

    prompt = base_prompt
    last_err = None
    for attempt in range(max_retries + 1):
        raw = call_llm(prompt)
        try:
            return parse_and_validate(raw)
        except Exception as e:
            last_err = str(e)
            prompt = (
                "REPAIR: Your previous output was invalid.\n"
                "Return ONLY JSON with keys person, company.\n"
                f"Invalid output:\n{raw}\n\n"
                f"Error:\n{last_err}\n"
            )

    raise ValueError(f"Failed after retries. Last error: {last_err}")


print(extract_with_repair("Ada Lovelace", call_llm_stub, max_retries=2))

In [ ]:
def extract_with_repair_todo(text: str, call_llm) -> dict:
    # TODO:
    # - Add raw-output persistence to output/llm_raw.txt on failure.
    # - Separate parse failures from schema failures in the error message.
    # - Add a retry cap parameter.
    raise NotImplementedError


print("Implement extract_with_repair_todo().")

## Common pitfalls

- Asking for JSON but not banning extra text
- Not separating parse failure vs schema failure
- No retry cap
- Mixing business logic with parsing/validation

## References

- Python `json`: https://docs.python.org/3/library/json.html
- Pydantic (optional): https://docs.pydantic.dev/latest/
- JSON Schema: https://json-schema.org/
- Tenacity: https://tenacity.readthedocs.io/

In [ ]:
def parse_and_validate(text: str) -> dict:
    data = parse_json_strict(text)
    validate_shape(data)
    return data


bad_outputs = [
    "Here is the JSON: {\"person\": \"Ada\", \"company\": null}",
    "{'person': 'Ada', 'company': null}",
    '{"person": "Ada"}',
]

for raw in bad_outputs:
    try:
        parse_and_validate(raw)
        print("OK", raw)
    except Exception as e:
        print("FAIL", type(e).__name__, "->", str(e))